# Отчет. Aнализ результатов A/B - теста для непрерывной метрики

In [1]:
# необходимые импорты

import pandas as pd
from scipy.stats import norm
from scipy.stats import ttest_ind

###  1.  Определение целевой метрики

Согласно условию проекта, целью проводимого теста
является снижение расходов на задание. На основании этого в качестве целевой выбираем метрику "Расходы" (и соответствующую графу датасета).

### 2. Альтернатива в критерии.

Обозначим:
f1 - тестовая выборка; f2 - контрольная выборка

Нулевая гипотеза Н0: m1 >= m2 (то есть внедрение предлагаемой системы не окажет статистически значимого влияния на среднюю величину расходов либо увеличит ее)

Альтернатива Н1: m1 < m2 (внедрение предлагаемой системы приведет к снижению средней величины расходов)

Рассматриваем одностороннюю альтернативу, так как согласно условию проекта цель теста состоит в том, чтобы установить факт именно снижения расходов на задание.

### 3. Расчет размера выборки

Размер выборки, с учетом того, что рассматриваем одностороннюю альтернативу, считаем по формуле

$$ n = m = \frac{2 \sigma^2 (z_{1-\alpha} - z_{\beta})^2}{d^2} $$


In [2]:
alpha = 0.05  # уровень значимости
z_alpha = norm.ppf(1 - alpha)
betta = 0.1   # по условию Мы хотели бы различать 5%-ное изменение целевой метрики с вероятностью 90%, т.е. 1-betta = 90/100
z_betta = norm.ppf(betta)

# параметры sigma и d считаем на основе исторических данных. поэтому импортируем их
df = pd.read_csv('hist_telesales.csv', header=0, sep=',')

d = 0.05 * df.Расходы.values.mean()   # равно mde, которое согласно вышеприведенному условию равно 5% от выборочного среднего целевой метрики
sigma = df.Расходы.values.std()   # выборочная дисперсия целевой метрики (из исторических данных)
n = 2 * (sigma**2) * ((z_alpha - z_betta)**2) / (d**2)
print(f'Минимальный размер выборки = {n}')

Минимальный размер выборки = 22530.66649491306


Принимаем размер выборки N = 22531. Именно такого размера выборки содержатся в тестовых и контрольных данных (файлы "Test.csv" и "Control.csv")

### 4. Принятие решения.

In [3]:
# считываем данные теста и контроля

df_test = pd.read_csv('test.csv', header=0, sep=',')
df_control = pd.read_csv('control.csv', header=0, sep=',')

cost_test = df_test.Расходы.values
cost_control = df_control.Расходы.values

Поскольку в рассматриваемой задаче мы проверяем гомогенность выборок по среднему, а дисперсии могут быть получены из данных выборок,
то в качестве критерия используем t-test.

Так как рассматриваем одностороннюю альтернативу, при которой среднее на тесте будет меньше чем среднее на контроле,
задаем параметр alternative='less'.

Устанавливаем параметр equal_var=False, так как дисперсии целевой метрики на тесте и контроле не равны.

В результате расчитываем статистику критерия и p-значение.

In [4]:
results = ttest_ind(cost_test, cost_control, equal_var=False, alternative='less')
print(f'Результаты расчета критерия: {results}')

Результаты расчета критерия: TtestResult(statistic=-2.30419387283739, pvalue=0.010608149523869117, df=44958.93566915675)


В рассматриваемом случае величина p-значения для тестовой и контрольной выборок оказалась вне диапазона \\( [\alpha, +\infty] \\) (для рассматриваемой задачи уровень значимости \\(\alpha = 0,05 \\)). Это является основнием для того, чтобы отвергнуть нулевую гиппотезу и принять альтернативную гиппотезу о том, что внедрение ситемы дозвона приведет к снижению расходов.